In [14]:
import tensorflow as tf
import numpy as np
import gradio as gr
from PIL import Image as im
import matplotlib.pyplot as plt

In [3]:
# load saved model from model.tflite file
interpreter = tf.lite.Interpreter(model_path="../model.tflite")
interpreter.allocate_tensors()

In [5]:
print(type(interpreter))
print(interpreter.get_input_details())
print(interpreter.get_output_details())

<class 'tensorflow.lite.python.interpreter.Interpreter'>
[{'name': 'serving_default_conv2d_78_input:0', 'index': 0, 'shape': array([  1, 224, 224,   3]), 'shape_signature': array([ -1, 224, 224,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 33, 'shape': array([1, 4]), 'shape_signature': array([-1,  4]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [45]:
# generate img jpgs from test set
test_images = np.load("../dataset/task7_X_test.npy")
test_labels = np.load("../dataset/task7_y_test.npy")

test_images = (test_images + 123.68)/274.75   # normalization
test_images *= 255
test_images = test_images.astype(np.uint8)

print(test_labels[0])
print(test_labels[130])
print(test_labels[260])
print(test_labels[390])

def save_image(np_array, filename):
    img = im.fromarray(np_array)
    img.convert("RGB").save(filename)

save_image(test_images[0], "heavy_damage.jpg")
save_image(test_images[130], "minor_damage.jpg")
save_image(test_images[260], "moderate_damage.jpg")
save_image(test_images[390], "undamaged.jpg")


[1. 0. 0. 0.]
[0. 1. 0. 0.]
[0. 0. 1. 0.]
[0. 0. 0. 1.]


In [78]:
from skimage.transform import resize

resized = resize(test_images[390], (224, 224))
save_image((resized*255).astype(np.uint8), "test.jpg")

In [79]:
from skimage.transform import resize

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

def classify_img(img):
    # preprocessing: resizing to appropriate shape, making pixel values between 0 and 1
    resized = resize(img, (224, 224))
    new_arr = resized.astype(np.float32)
    #print(resized.shape)
    #new_arr = resized.astype(np.float32)
    #new_arr /= 255

    interpreter.set_tensor(input_details[0]['index'], [new_arr])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    results = {}
    results["Heavy damage"] = output_data[0][0]
    results["Minor damage"] = output_data[0][1]
    results["Moderate damage"] = output_data[0][2]
    results["Undamaged damage"] = output_data[0][3]
    return results

print(classify_img(test_images[0]))

{'Heavy damage': 0.4104678, 'Minor damage': 0.07655419, 'Moderate damage': 0.4881688, 'Undamaged damage': 0.024809109}


In [82]:
# creating demo with Gradio
demo = gr.Interface(fn=classify_img, inputs=gr.Image(type="numpy"), outputs="label")
demo.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
